In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['embeddings', 'train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
np.random.seed(7)

train_df = pd.read_table('../input/train.csv',sep=',')
test_df = pd.read_table('../input/test.csv',sep=',')

#full_df = pd.concat([train_df[,1], test_df[,1]])
print(train_df.shape, test_df.shape)

(1306122, 3) (56370, 2)


In [3]:
full_df = pd.concat([train_df["question_text"], test_df["question_text"]])
#print(full_df)
raw_text=full_df.str.lower()

#full_df=full_df.values.reshape(-1,1)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(full_df)
true_k = 7
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=300, n_init=1)
model.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=7, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [5]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

print("\n")
print("Prediction")


Top terms per cluster:
Cluster 0:
 like
 did
 feel
 does
 look
 life
 don
 world
 start
 just
Cluster 1:
 people
 india
 best
 like
 does
 don
 white
 quora
 hate
 black
Cluster 2:
 does
 best
 good
 life
 time
 way
 use
 work
 world
 quora
Cluster 3:
 think
 people
 does
 trump
 don
 like
 world
 good
 women
 quora
Cluster 4:
 know
 don
 people
 want
 does
 need
 things
 like
 person
 love
Cluster 5:
 make
 money
 does
 people
 way
 feel
 good
 like
 friends
 best
Cluster 6:
 companies
 mobile
 space
 best
 app
 phone
 india
 does
 number
 work


Prediction


In [7]:
KY = vectorizer.transform(train_df["question_text"])
prediction = model.predict(KY)
KYT = vectorizer.transform(test_df["question_text"])
prediction_test = model.predict(KYT)
prediction=prediction.reshape(-1,1)
prediction_test=prediction_test.reshape(-1,1)
print(prediction.shape)
print(prediction_test.shape)
print("DONE")

DONE


In [29]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(prediction)
predklabel = le.transform(prediction)
predklabelTest = le.transform(prediction_test)
print(predklabel.shape)
print(predklabel)

(1306122,)
[0 1 6 ... 2 2 2]


In [8]:
from keras.preprocessing.text import Tokenizer
tok_raw = Tokenizer()
tok_raw.fit_on_texts(raw_text)
print("DONE")

Using TensorFlow backend.


DONE


In [ ]:
train.max()

In [ ]:
np.array(train).max()

In [ ]:
np.max(train_df.question_text.str.len().max())

In [66]:
from keras.preprocessing.sequence import pad_sequences
train=tok_raw.texts_to_sequences(train_df.question_text.str.lower())
test=tok_raw.texts_to_sequences(test_df.question_text.str.lower())


MAX_TEXT = np.max([
    np.max(np.array(train).max()),
    np.max(np.array(test).max()),
]) + 100
#MAX_TEXT=np.max(train.max())+100
MAX_ITEM_DESC_SEQ=255
MAX_CATEGORY = np.max(predklabel.max()) + 1
y_train=train_df.target
print(MAX_TEXT)
print(MAX_CATEGORY)

227633
7


In [67]:
x_train=pad_sequences(train, maxlen=MAX_ITEM_DESC_SEQ),
print(x_train[0].shape)

x_test=pad_sequences(test, maxlen=MAX_ITEM_DESC_SEQ),
print(x_test[0].shape)

(1306122, 255)
(56370, 255)


In [ ]:
print(x_train.shape)

(1306122, 1)
(56370, 1)


In [68]:
from sklearn.model_selection import train_test_split
XSeq,XTSeq,XK,XTK,Y,YT=train_test_split(x_train[0],predklabel,y_train,test_size=0.08,stratify=y_train)

XK=XK.reshape(-1,1)
XTK=XTK.reshape(-1,1)
#Y=Y.reshape(-1,1)
#YT=YT.reshape(-1,1)
print(XSeq.shape)
print(XK.shape)
print(Y.shape)

(1201632, 255)
(1201632, 1)
(1201632,)


In [ ]:

np.random.seed(7)
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers import Conv1D,Input, Dropout, Dense, concatenate, GRU, Embedding, Flatten, Activation,BatchNormalization,GaussianNoise
from keras.layers import Input, Dropout, Dense, concatenate, GRU, Embedding, Flatten, Activation,BatchNormalization,GaussianNoise,GaussianDropout
from keras.layers import Bidirectional
from keras.optimizers import Adam,SGD
from keras.models import Model
from keras import backend as K



def noise_measurement(train, noise_level):
    noised_train = train.copy()
    to_transform = np.random.random(train.shape) &lt; noise_level
    transform_y = np.random.randint(0, max_features, size=train.shape)
    noised_train[to_transform] = transform_y[to_transform]
    return noised_train

def get_model(lr=0.001, decay=0.0):    
    item_desc = Input(shape=[XSeq.shape[1]], name="item_desc")  
    #item_desc_gau=Dropout(0.3)(item_desc)
    Klabel= Input(shape=[1], name="klabel")      
    emb_category_name = Embedding(MAX_CATEGORY, 10)(Klabel)    
    emb_item_desc = Embedding(MAX_TEXT, 120)(item_desc)            
    rnn_layer1 = GRU(50,return_sequences=True) (emb_item_desc)
    rnn_layer2=GRU(25)(rnn_layer1)
    
    main_l = concatenate([
        Flatten() (emb_category_name)
        #       
        , rnn_layer2
#         , rnn_layer3
        
    ])
   # main_l = Dropout(0.6)(Dense(512,kernel_initializer='normal',activation='relu') (main_l))     
   # main_l = Dropout(0.6)(Dense(256,kernel_initializer='normal',activation='relu') (main_l))    
   # main_l = Dropout(0.6)(Dense(128,kernel_initializer='normal',activation='relu') (main_l))    
   # main_l = Dropout(0.6)(Dense(64,kernel_initializer='normal',activation='relu') (main_l))    
    output = Dense(1, activation="sigmoid") (main_l)
    optimizer = Adam(lr=lr, decay=decay)
    model = Model([item_desc,Klabel], output)
    #sparse_categorical_crossentropy
    
    model.compile(loss ="binary_crossentropy", optimizer = optimizer,metrics=['acc'])
    model.summary()
    return model


# Set hyper parameters for the model.
BATCH_SIZE = 320
epochs = 2

# Calculate learning rate decay.
exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1
steps = int(len(x_train[0]) / BATCH_SIZE) * epochs
lr_init, lr_fin = 0.005, 0.001
lr_decay = exp_decay(lr_init, lr_fin, steps)

#F1_Score = F1Evaluation(validation_data=([XTSeq,XTK], YT), interval=1)

model = get_model(lr=lr_init, decay=lr_decay)
model.fit(
        [XSeq,XK],Y, epochs=epochs, batch_size=BATCH_SIZE, verbose=1,validation_data=([XTSeq,XTK],YT)
    #,callbacks=[F1_Score]
)


In [62]:
from sklearn import metrics

pred_val_y_4 = model.predict([XTSeq,XTK], batch_size=1024, verbose=1)

104490/104490 [==============================] - 17s 166us/step


In [63]:

thresholds = []
for thresh in np.arange(0.01, 0.99, 0.01):
    thresh = np.round(thresh, 2)
    res = metrics.f1_score(YT, (pred_val_y_4 > thresh).astype(int))
    thresholds.append([thresh, res])
    print("F1 score at threshold {0} is {1}".format(thresh, res))
    
thresholds.sort(key=lambda x: x[1], reverse=True)
best_thresh_4 = thresholds[0][0]
print("Best threshold: ", best_thresh_4)

#y_pred_4 = MODEL4.predict(x_test, batch_size=1024, verbose=True)




F1 score at threshold 0.01 is 0.1330481470164564
F1 score at threshold 0.02 is 0.136654168898411
F1 score at threshold 0.03 is 0.13703207177477572
F1 score at threshold 0.04 is 0.13747043272575482
F1 score at threshold 0.05 is 0.1378088242847604
F1 score at threshold 0.06 is 0.13789429412589374
F1 score at threshold 0.07 is 0.13864997112725286
F1 score at threshold 0.08 is 0.1389578163771712
F1 score at threshold 0.09 is 0.13856683894398658
F1 score at threshold 0.1 is 0.1386131251161926
F1 score at threshold 0.11 is 0.13733736881411895
F1 score at threshold 0.12 is 0.13753796230810833
F1 score at threshold 0.13 is 0.1357224217997021
F1 score at threshold 0.14 is 0.1342128166368925
F1 score at threshold 0.15 is 0.132485781197725
F1 score at threshold 0.16 is 0.13145820865749053
F1 score at threshold 0.17 is 0.13075687716935191
F1 score at threshold 0.18 is 0.12879276952872823
F1 score at threshold 0.19 is 0.1277166322140902
F1 score at threshold 0.2 is 0.12643146327356852
F1 score at t

In [64]:
y_test=model.predict([x_test[0],predklabelTest], batch_size=1024, verbose=1)
y_te = (y_test[:,0] > best_thresh_4).astype(np.int)

submit_df = pd.DataFrame({"qid": test_df["qid"], "prediction": y_te})
submit_df.to_csv("submission.csv", index=False)


56370/56370 [==============================] - 9s 153us/step


In [65]:
submit_df

,qid,prediction
0,00014894849d00ba98a9,0
1,000156468431f09b3cae,1
2,000227734433360e1aae,0
3,0005e06fbe3045bd2a92,0
4,00068a0f7f41f50fc399,0
5,000a2d30e3ffd70c070d,0
6,000b67672ec9622ff761,0
7,000b7fb1146d712c1105,0
8,000d665a8ddc426a1907,0
9,000df6fd2229447b2969,1
